JavaScript in GitHub Codespaces (Node.js) — A Hands-On Jupyter Module

# JavaScript in Codespaces (Node.js) — Jupyter Learning Module

## What you’ll learn
- Run JavaScript (Node.js) from a Jupyter notebook in Codespaces
- Core JavaScript: variables, types, strings, arrays, objects, functions
- Modern JS: modules, async/await, fetch, JSON
- File I/O and small data tasks (CSV-style parsing, summary stats)
- Mini project: read a dataset and produce a simple report

## Prereqs
- Basic programming comfort (loops, variables)
- You do NOT need a special JavaScript kernel — we’ll run Node from cells.


In [2]:
!node -v
!npm -v


v18.20.4


9.2.0


## Running JavaScript from this notebook

We’ll use Node.js via shell commands:

- `node -e "JS CODE HERE"` for quick one-liners
- `node my_script.mjs` for real scripts
- We'll write `.mjs` files from notebook cells, then run them.

Why `.mjs`? It enables modern ES Modules (`import/export`) without extra config.


In [3]:
!node -e "console.log('Hello from Node in Codespaces!')"


Hello from Node in Codespaces!


## A1. Variables & types

JavaScript has:
- `let` (reassignable)
- `const` (not reassignable)
- `var` (older; avoid in new code)

Types you’ll use a lot:
- number, string, boolean, null, undefined
- object (includes arrays)


In [4]:
!node -e "let age=42; const name='Pat'; const active=true; console.log({age, name, active, t_age: typeof age, t_name: typeof name});"


{
  age: 42,
  name: 'Pat',
  active: true,
  t_age: 'number',
  t_name: 'string'
}


## A2. Strings & template literals

Use backticks for interpolation:
`Hello, ${name}`


In [5]:
!node -e "const name='Taylor'; const steps=8432; console.log(`Hello, ${name}. Steps today: ${steps}.`);"


/bin/bash: line 1: Hello,: command not found



## B1. Arrays

Common operations:
- `push`, `map`, `filter`, `reduce`
- `length`


In [6]:
!node -e "const bp=[120, 135, 128, 142]; const high=bp.filter(x=>x>=140); const avg=bp.reduce((a,b)=>a+b,0)/bp.length; console.log({bp, high, avg});"


{ bp: [ 120, 135, 128, 142 ], high: [ 142 ], avg: 131.25 }


## B2. Objects

Objects are key-value maps.


In [7]:
!node -e "const patient={id:101, name:'Ari', meds:['statin','acei'], vitals:{sbp:132, dbp:84}}; console.log(patient); console.log('SBP:', patient.vitals.sbp);"


{
  id: 101,
  name: 'Ari',
  meds: [ 'statin', 'acei' ],
  vitals: { sbp: 132, dbp: 84 }
}
SBP: 132


## C1. Functions

Two common styles:
- Function declaration: `function add(a,b){...}`
- Arrow function: `const add=(a,b)=>a+b`


In [8]:
!node -e "function bmiKgM2(kg,m){return kg/(m*m)}; const bmi=(kg,m)=>kg/(m*m); console.log(bmiKgM2(80,1.75), bmi(80,1.75));"


26.122448979591837 26.122448979591837


## C2. Write a script file

We'll create `01_basics.mjs` and run it.


In [9]:
code = r"""
// 01_basics.mjs
const readings = [120, 135, 128, 142, 118];

const mean = arr => arr.reduce((a,b)=>a+b,0)/arr.length;

const summary = {
  n: readings.length,
  min: Math.min(...readings),
  max: Math.max(...readings),
  mean: mean(readings),
  highCount: readings.filter(x => x >= 140).length
};

console.log("BP Summary:", summary);
"""
with open("01_basics.mjs","w") as f:
    f.write(code)

!node 01_basics.mjs


BP Summary: { n: 5, min: 118, max: 142, mean: 128.6, highCount: 1 }


Modules + Packages (npm) in Codespaces

## D1. Initialize a Node project

This creates `package.json` so you can install packages.


In [10]:
!npm init -y


Wrote to /workspaces/Data_Science-notebooks/lessons/Jupyter/package.json:

{
  "name": "jupyter",
  "version": "1.0.0",
  "description": "",
  "main": "index.js",
  "scripts": {
    "test": "echo \"Error: no test specified\" && exit 1"
  },
  "keywords": [],
  "author": "",
  "license": "ISC"
}




## D2. Install a package

We'll use `papaparse` to parse CSV easily.


In [11]:
!npm install papaparse


m##################) ⠼ reify:papaparse: http fetch GET 200 https://registry.npm[
added 1 package, and audited 2 packages in 377ms

found 0 vulnerabilities


Mini data task: Parse CSV + compute summary

In [ ]:
## E1. Create a small CSV dataset

We'll create `bp_sample.csv` with id, sbp, dbp.


In [12]:
csv_text = """id,sbp,dbp
1,120,78
2,142,90
3,135,85
4,128,82
5,155,95
"""
with open("bp_sample.csv","w") as f:
    f.write(csv_text)

!ls -lh bp_sample.csv


-rw-rw-rw- 1 vscode vscode 56 Feb  8 03:40 bp_sample.csv


## E2. Parse CSV and compute summary stats

We'll compute:
- Mean SBP / DBP
- Count of SBP >= 140
- A simple “stage” label per row (illustrative, not clinical advice)


In [13]:
code = r"""
// 02_csv_report.mjs
import fs from "fs";
import Papa from "papaparse";

const text = fs.readFileSync("bp_sample.csv", "utf8");
const parsed = Papa.parse(text, { header: true, dynamicTyping: true });

const rows = parsed.data.filter(r => r.id != null);

const mean = (arr) => arr.reduce((a,b)=>a+b,0)/arr.length;

const sbps = rows.map(r => r.sbp);
const dbps = rows.map(r => r.dbp);

const report = {
  n: rows.length,
  mean_sbp: mean(sbps),
  mean_dbp: mean(dbps),
  high_sbp_count: sbps.filter(x => x >= 140).length,
};

function labelBP(sbp, dbp){
  if (sbp >= 140 || dbp >= 90) return "High (example)";
  if (sbp >= 130 || dbp >= 80) return "Elevated (example)";
  return "Normal (example)";
}

const labeled = rows.map(r => ({...r, label: labelBP(r.sbp, r.dbp)}));

console.log("Report:", report);
console.log("Labeled rows:");
console.table(labeled);
"""
with open("02_csv_report.mjs","w") as f:
    f.write(code)

!node 02_csv_report.mjs


Report: { n: 5, mean_sbp: 136, mean_dbp: 86, high_sbp_count: 2 }
Labeled rows:
┌─────────┬────┬─────┬─────┬──────────────────────┐
│ (index) │ id │ sbp │ dbp │        label         │
├─────────┼────┼─────┼─────┼──────────────────────┤
│    0    │ 1  │ 120 │ 78  │  'Normal (example)'  │
│    1    │ 2  │ 142 │ 90  │   'High (example)'   │
│    2    │ 3  │ 135 │ 85  │ 'Elevated (example)' │
│    3    │ 4  │ 128 │ 82  │ 'Elevated (example)' │
│    4    │ 5  │ 155 │ 95  │   'High (example)'   │
└─────────┴────┴─────┴─────┴──────────────────────┘


Async/Await + Fetch (API call)

## F1. Async/Await + fetch

Node supports `fetch` in modern versions.
We’ll fetch a public JSON endpoint and print a field.

If your environment blocks outbound network calls, this may fail—if so, skip it.


In [14]:
code = r"""
// 03_fetch_demo.mjs
const url = "https://api.github.com/repos/nodejs/node";
const res = await fetch(url, { headers: { "User-Agent": "codespaces-demo" } });
if (!res.ok) throw new Error(`HTTP ${res.status}`);
const data = await res.json();
console.log({ full_name: data.full_name, stars: data.stargazers_count, updated_at: data.updated_at });
"""
with open("03_fetch_demo.mjs","w") as f:
    f.write(code)

!node 03_fetch_demo.mjs


{
  full_name: 'nodejs/node',
  stars: 115597,
  updated_at: '2026-02-07T23:03:43Z'
}


Exercises (students can do these)

In [16]:
code = r"""
// bp_report.mjs
import fs from "fs";
import Papa from "papaparse";

const filename = process.argv[2];
if (!filename) {
  console.error("Usage: node bp_report.mjs <file.csv>");
  process.exit(1);
}

const text = fs.readFileSync(filename, "utf8");
const parsed = Papa.parse(text, { header: true, dynamicTyping: true });
const rows = parsed.data.filter(r => r.id != null);

const mean = (arr) => arr.reduce((a,b)=>a+b,0)/arr.length;

const sbps = rows.map(r => r.sbp).filter(x => Number.isFinite(x));
const dbps = rows.map(r => r.dbp).filter(x => Number.isFinite(x));

const report = {
  file: filename,
  n: rows.length,
  mean_sbp: sbps.length ? mean(sbps) : null,
  mean_dbp: dbps.length ? mean(dbps) : null,
  high_sbp_count: sbps.filter(x => x >= 140).length,
};

fs.writeFileSync("report.json", JSON.stringify(report, null, 2));
console.log("Wrote report.json");
console.log(report);
"""
with open("bp_report.mjs","w") as f:
    f.write(code)

!node bp_report.mjs bp_sample.csv
!cat report.json


Wrote report.json
{
  file: 'bp_sample.csv',
  n: 5,
  mean_sbp: 136,
  mean_dbp: 86,
  high_sbp_count: 2
}
{
  "file": "bp_sample.csv",
  "n": 5,
  "mean_sbp": 136,
  "mean_dbp": 86,
  "high_sbp_count": 2
}